# Figure out area

Trying to see if I need to convert TAREA before working out Gg carbon for sDIC decomposition

In [24]:
import numpy as np
import pandas as pd
import xarray as xr
import esmtools as et
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
def detect_EBUS():
    lat1 = 25
    lat2 = 46
    lon1 = 215
    lon2 = 260
    return lat1, lat2, lon1, lon2

In [11]:
ds = xr.open_dataset('/glade/scratch_new/rbrady/EBUS_BGC_Variability/' +
                     'SST_monthly/SST.002.192001-210012.nc',
                     decode_times=False)

In [12]:
ds.coords['time'] = pd.date_range('1920-01', '2101-01', freq='M')
ds.attrs = ''
ds = ds.squeeze()

In [13]:
ds['TAREA'] = ds['TAREA'] / (100 * 100)

In [14]:
lat1, lat2, lon1, lon2 = detect_EBUS()

In [15]:
ds = ds.sel(time=slice('1920-01', '2015-12'))

In [16]:
a, c = et.filtering.find_indices(ds['TLAT'].values, ds['TLONG'].values, lat1,
                                 lon1)
b, d = et.filtering.find_indices(ds['TLAT'].values, ds['TLONG'].values, lat2,
                                 lon2)

In [25]:
ds = ds.isel(nlat=slice(a, b), nlon=slice(c, d))

# extract domain

In [30]:
OFFSHORE = 800
ds['DXT'] = ds['DXT'] / 100 / 1000

In [32]:
lat1, lat2 = et.ebus.latitude_bounds('CalCS')

In [33]:
ds = ds.where(ds['TLAT'] >= lat1).where(ds['TLAT'] <= lat2)

In [34]:
data = ds['SST'].isel(time=0)
data = np.ma.array(data, mask=np.isnan(data))
dxt_dat = ds['DXT']
dxt_dat = np.ma.array(dxt_dat, mask=np.isnan(data))
ds['DXT'] = (('nlat', 'nlon'), dxt_dat)

In [35]:
regmask = ds['REGION_MASK']
counter = 0
for row in regmask:
    conditional = 0 in row.values
    if conditional == False:
        ds['DXT'][counter, :] = np.nan
    counter += 1

In [36]:
x = ds['DXT'].values
x = np.ma.array(x, mask=np.isnan(x))
dxt_cum = np.cumsum(x[:, ::-1], axis=1)[:, ::-1]
ds['DXT_Cum'] = (('nlat', 'nlon'), dxt_cum)

In [37]:
ds = ds.where(ds['DXT_Cum'] <= OFFSHORE)

# Actually compare area

In [43]:
ds['TAREA'].sum()

<xarray.DataArray 'TAREA' ()>
array(7.760382e+11)
Coordinates:
    z_t      float32 500.0

In [44]:
EBU = 'CalCS'
# Convert to TgC / yr
filepath = ('/glade/work/rbrady/EBUS_BGC_Variability/FG_CO2/' +
            EBU + '/filtered_output/')
filename = (EBU.lower() + '-FG_CO2-forced-signal-chavez-800km.nc')
area = xr.open_dataset(filepath + filename)
area = area['TAREA']


# Total area
total_area = area.sum()

In [45]:
total_area

<xarray.DataArray 'TAREA' ()>
array(7.760382e+11)